In [1]:
import pandas as pd
import numpy as np
import math
import os
import spotipy

from IPython.display import display
pd.options.display.max_columns = None

This is a feature engineering script that generates variables that can be inferred from the Spotify Top 200 Charts in the US.

### Import raw Spotify charts data

We import the raw dataset of all songs on the Spotify Top 200 Charts since 2017 (assembled by Brunnell), regardless if they feature a collaboration or not. This is because, to generate some artist-specific variables on the number of appearances/songs on the chart, we need to have a complete history of each artist, including all the songs they have released without collaborations.

In [2]:
data_top200 = pd.read_csv('../input/spotify_top200.csv')\
.drop_duplicates()
data_top200['Date'] = pd.to_datetime(data_top200['Date'])

In [3]:
data_us = data_top200[data_top200['Country']=='United States']\
.sort_values(by=['Artist', 'Date', 'Track Name'])
data_us['Track URI'] = data_us['Track URL'].str.replace(r'https://open.spotify.com/track/', '')

Conduct a few sanity checks to make sure the date range is complete.

In [4]:
print(max(data_us['Date']))
print(min(data_us['Date']))

2020-12-07 00:00:00
2017-01-01 00:00:00


Check that 200 unique positions per day x 1404 unique days (between 2017-01-01 and 2020-12-07) = 280600 observations in the US.

In [5]:
print(len(data_us.index))

x = data_us.groupby('Date')[['Position']].transform('count')
print(np.max(x))
print(np.min(x))

y = data_us.groupby('Position')[['Date']].transform('count')
print(np.max(y))
print(np.min(y))

280600
Position    200
dtype: int64
Position    200
dtype: int64
Date    1403
dtype: int64
Date    1403
dtype: int64


### Setting up Spotipy Credentials

In [6]:
creds = pd.read_csv('../input/credentials.csv')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]

client_credentials_manager = spotipy.SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, 
                                                              client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Generate variables at a song-level (collaborators involved, release date)

In [7]:
track_uris = data_us['Track URI'].unique()
len(track_uris)

7313

For each unique track in data_us, download the list of artists/collaborators involved, their names, album id, and their release date.

In [8]:
def get_track_data(track_uris, n):
    
    track_artist_uris = []
    track_artist_names = []
    track_album_uri = []
    track_album_release = []
    loops = 0
    
#     while n* (loops+1) <= len(track_uris):
    for loops in range(math.ceil(len(track_uris)/n)):
        # getting track information in batches of size n
        temp = [x for x in sp.tracks(track_uris[n*loops : n*(loops + 1)])['tracks']]
    
        # for each track in this batch, obtain album info and list of artists
        for i in range(len(temp)):
            track_album_uri.append(temp[i]['album']['id'])
            track_album_release.append(temp[i]['album']['release_date'])
        
            t = temp[i]['artists']
            temp_uris = []
            temp_names = []
        
            # for each artist, obtain id and name
            for j in range(len(t)):
                temp_uris.append(t[j]['id']) 
                temp_names.append(t[j]['name'])
                
            track_artist_uris.append(temp_uris)
            track_artist_names.append(temp_names)
        
        loops += 1
        print(f'Found information for {n*(loops+1)} songs')
    
    # Combine all lists into a dataframe
    track_data = pd.DataFrame(list(zip(track_uris, track_artist_uris, track_artist_names,
                                   track_album_uri, track_album_release)),
                              columns=['Track URI', 'Collaborator URI', 'Collaborator Name',
                                       'Album URI', 'Album Release'])
    
    # Count number of collaborators per song 
    track_data['No. of Collaborators'] = track_data['Collaborator URI'].astype(str).str.count(',')+1 
    return(track_data)

In [9]:
track_data = get_track_data(track_uris, n=50)
# track_data = pd.read_csv('../output/2021.01.01 Track Artist URI 7313.csv')\
# .drop(columns = ['Unnamed: 0'])

Found information for 100 songs
Found information for 150 songs
Found information for 200 songs
Found information for 250 songs
Found information for 300 songs
Found information for 350 songs
Found information for 400 songs
Found information for 450 songs
Found information for 500 songs
Found information for 550 songs
Found information for 600 songs
Found information for 650 songs
Found information for 700 songs
Found information for 750 songs
Found information for 800 songs
Found information for 850 songs
Found information for 900 songs
Found information for 950 songs
Found information for 1000 songs
Found information for 1050 songs
Found information for 1100 songs
Found information for 1150 songs
Found information for 1200 songs
Found information for 1250 songs
Found information for 1300 songs
Found information for 1350 songs
Found information for 1400 songs
Found information for 1450 songs
Found information for 1500 songs
Found information for 1550 songs
Found information for 1600 s

In [10]:
# track_data.to_csv('../output/2021.01.01 Track Artist URI 7313.csv')

Generate variables on the timing of the song's release. 

In [11]:
track_data['Album Release'] = pd.to_datetime(track_data['Album Release'])
track_data['Album_release_month'] = track_data['Album Release'].dt.strftime('%m')
track_data['Album_release_dayweek'] = track_data['Album Release'].dt.strftime('%a')

In [12]:
track_data

,Track URI,Collaborator URI,Collaborator Name,Album URI,Album Release,No. of Collaborators,Album_release_month,Album_release_dayweek
0,4nutwPQrK56fFmrAMgyPhz,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
1,7v2azTfke2BR57lh2HxPQo,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
2,0fyBYsrmpihh1mfalssDlB,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
3,6VFKlX5qzxwmIiezqeqNYG,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
4,53AiGAa0Qi2VbX7eUpur1U,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
...,...,...,...,...,...,...,...,...
7308,3RXkboS74UYzN14xTqzPyY,[6IhKl7lqJc5omkhODEJinj],[],4yQC726OERjevM2YKtORnm,2017-07-14,1,07,Fri
7309,3bVbQvGVIe4n24AzyXovXh,[3h1fFIofdTGrHbqisHyWgI],[],6zP4EpOz9M0vhrjp7FNwKN,2017-07-17,1,07,Mon
7310,4JAyIDXOqNM6qHuZML01uX,[3h1fFIofdTGrHbqisHyWgI],[],6zP4EpOz9M0vhrjp7FNwKN,2017-07-17,1,07,Mon
7311,6Br5mChPdgQNmLF0G0gjPH,[6IrDpI3xcuzTUiEc3fnc9H],[],3jd8KzVKriW0uzSsIwxfZM,2017-07-18,1,07,Tue


Check that no songs were missed from the original list of track_uris.

In [13]:
setdiff = np.setdiff1d(track_uris, track_data['Track URI'])
len(setdiff)

0

Confirm that there are 2843 songs with collaborations, as Brunnell has found previously.

In [14]:
bru = pd.read_csv('../input/spotify_top200_reduced.csv').drop_duplicates()
bru_us = bru[bru['Country']=='United States']

uri1 = track_data.loc[track_data['No. of Collaborators']!= 1,'Track URI'].drop_duplicates()
uri2 = bru_us['Track URI'].drop_duplicates()
print(len(uri1))
print(len(uri2))
print(len(np.setdiff1d(uri1, uri2)))
print(len(np.intersect1d(uri1, uri2)))

2843
2843
0
2843


Merge data_us onto track_data

In [15]:
data1 = data_us.merge(track_data, how = 'left', on = 'Track URI')
data1

,Date,Track URL,Position,Track Name,Artist,Streams,Country,Track URI,Collaborator URI,Collaborator Name,Album URI,Album Release,No. of Collaborators,Album_release_month,Album_release_dayweek
0,2018-09-07,https://open.spotify.com/track/4nutwPQrK56fFmr...,196,"10,000 Degrees",$uicideBoy$,232152,United States,4nutwPQrK56fFmrAMgyPhz,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
1,2018-09-07,https://open.spotify.com/track/7v2azTfke2BR57l...,177,Bring Out Your Dead,$uicideBoy$,247359,United States,7v2azTfke2BR57lh2HxPQo,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
2,2018-09-07,https://open.spotify.com/track/0fyBYsrmpihh1mf...,146,King Tulip,$uicideBoy$,270299,United States,0fyBYsrmpihh1mfalssDlB,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
3,2018-09-07,https://open.spotify.com/track/6VFKlX5qzxwmIie...,190,Meet Mr. NICEGUY,$uicideBoy$,236054,United States,6VFKlX5qzxwmIiezqeqNYG,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
4,2018-09-07,https://open.spotify.com/track/53AiGAa0Qi2VbX7...,140,Nicotine Patches,$uicideBoy$,279299,United States,53AiGAa0Qi2VbX7eUpur1U,[1VPmR4DJC1PlOtd0IADAO0],[$uicideBoy$],72I2i7wwU3Q7mJGxbNW12D,2018-09-07,1,09,Fri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280595,2017-11-10,https://open.spotify.com/track/1YqcGlCHNquxBhl...,182,NaN,NaN,205945,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue
280596,2017-11-11,https://open.spotify.com/track/1YqcGlCHNquxBhl...,151,NaN,NaN,200095,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue
280597,2017-11-12,https://open.spotify.com/track/1YqcGlCHNquxBhl...,160,NaN,NaN,176687,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue
280598,2017-11-13,https://open.spotify.com/track/1YqcGlCHNquxBhl...,170,NaN,NaN,190496,United States,1YqcGlCHNquxBhlUZsjhMT,[2uwXb02RFgtGOr6s6XMWlB],[],2oxjmU5c2mZzc7mSQqWTE9,2017-11-07,1,11,Tue


### Generate variables that change on a song-day basis

**Song_first_onchart**: a song's earliest appearance on the chart.

**Song_days_since_first**: the number of days between the current chart day and the song's first day on the chart.

**Song_days_onchart**: the actual number of days that the song has appeared on the chart.

**Song_days_since_release**: the number of days between the current chart day and the song's album's release date.

**Date_diff**: the number of days elapsed between a song's current appearance and its last appearance If Date_diff = 1, then the song appeared yesterday and today.

**Song_new_streak**: when Date_diff is > 1 , we assume that the song has temporarily dropped off the chart (hence ending its last consecutive streak), so we create a binary variable "Song_new_streak" that signals the beginnning of a new streak.

**Song_streak_id**: for each song's consecutive streak, assign a unique id by performing a cumulative sum of the 'Song_new_streak' variable.

**Song_consec_day**: for each consecutive day within a streak, assign a unique id.

In [16]:
track_days = data1[['Date', 'Track URI', 'Album Release']]\
.drop_duplicates().sort_values(by = ['Track URI', 'Date'])

track_days['Year_chart'] = track_days['Date'].dt.strftime('%Y').astype(int)
track_days['Month_chart'] = track_days['Date'].dt.strftime('%m')
track_days['Song_first_onchart'] = track_days.groupby(['Track URI'])[['Date']].transform('min')
track_days['Song_days_since_first'] = track_days['Date'] - track_days['Song_first_onchart']
track_days['Song_days_since_first'] = track_days['Song_days_since_first'].apply(lambda x: x.days)
track_days['Song_days_onchart'] = track_days.groupby(['Track URI']).cumcount() + 1
track_days['Song_days_since_release'] = track_days['Date'] - track_days['Album Release']
track_days['Song_days_since_release'] = track_days['Song_days_since_release'].apply(lambda x: x.days)

track_days['Date_lag'] = track_days['Date'].shift(1)
track_days['Date_diff'] = track_days['Date'] - track_days['Date_lag']
track_days['Date_diff'] = track_days['Date_diff'].apply(lambda x: x.days)
track_days['Song_new_streak'] = np.where((track_days['Song_days_onchart']==1)|
                                         (track_days['Date_diff']>1), 
                                         1, 0)
track_days['Song_streak_id'] = track_days.groupby(['Track URI'])[['Song_new_streak']].transform('cumsum')
track_days['Song_consec_day'] = track_days.groupby(['Track URI', 'Song_streak_id']).cumcount() + 1

track_days

,Date,Track URI,Album Release,Year_chart,Month_chart,Song_first_onchart,Song_days_since_first,Song_days_onchart,Song_days_since_release,Date_lag,Date_diff,Song_new_streak,Song_streak_id,Song_consec_day
275995,2017-03-24,000xQL6tZNLJzIrtIgxqSl,2017-03-23,2017,03,2017-03-24,0,1,1,NaT,NaN,1,1,1
275997,2017-03-25,000xQL6tZNLJzIrtIgxqSl,2017-03-23,2017,03,2017-03-24,1,2,2,2017-03-24,1.0,0,1,2
275999,2017-03-26,000xQL6tZNLJzIrtIgxqSl,2017-03-23,2017,03,2017-03-24,2,3,3,2017-03-25,1.0,0,1,3
276001,2017-03-27,000xQL6tZNLJzIrtIgxqSl,2017-03-23,2017,03,2017-03-24,3,4,4,2017-03-26,1.0,0,1,4
276003,2017-03-28,000xQL6tZNLJzIrtIgxqSl,2017-03-23,2017,03,2017-03-24,4,5,5,2017-03-27,1.0,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187603,2017-10-10,7zxRMhXxJMQCeDDg0rKAVo,2017-02-24,2017,10,2017-02-16,236,171,228,2017-09-28,12.0,1,22,1
187605,2017-10-11,7zxRMhXxJMQCeDDg0rKAVo,2017-02-24,2017,10,2017-02-16,237,172,229,2017-10-10,1.0,0,22,2
187614,2017-10-17,7zxRMhXxJMQCeDDg0rKAVo,2017-02-24,2017,10,2017-02-16,243,173,235,2017-10-11,6.0,1,23,1
187617,2017-10-18,7zxRMhXxJMQCeDDg0rKAVo,2017-02-24,2017,10,2017-02-16,244,174,236,2017-10-17,1.0,0,23,2


## Convert list of collaborator URIs and names into long format

For each song/track URI, we expand/explode the list of Collaborator URI and Collaborator Name so that each artist can be on a separate row. This allows us to merge the characteristics of each artist later on. 

In [17]:
artist_uri = data1.drop(columns = ['Collaborator Name', 'Track URL', 'Artist', 'Album Release'])
artist_uri2 = artist_uri.explode('Collaborator URI', ignore_index=True)
artist_uri2[artist_uri2['No. of Collaborators']!=1]

,Date,Position,Track Name,Streams,Country,Track URI,Collaborator URI,Album URI,No. of Collaborators,Album_release_month,Album_release_dayweek
182,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,6PfSUFtkMVoDkx4MQkzOi3,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
183,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,4UXqAaa6dQYAk18Lv7PEgX,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
184,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,5aYf0AInMznHfXGaemKEBv,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
185,2020-07-10,182,hand crushed by a mallet (Remix) [feat. Fall O...,238674,United States,5Mm2CJzNRiICC5MWRWQnBo,0MfC3pip8rY8OFLJVVNvBO,0qnExDZfz0kVeBjixPsyjS,4,07,Fri
186,2017-01-01,187,Good Drank,146863,United States,39pS70eeDvyCAF3t8NAlVV,17lzZA2AlOHwCwFALHttmp,5vvvo79z68vWj9yimoygfS,3,06,Fri
...,...,...,...,...,...,...,...,...,...,...,...
437644,2020-10-01,192,Prospect (ft. Lil Baby),210504,United States,4iHSE5R1U8jf84tRn52xRt,5f7VJjfbwm532GiveGC0ZK,4Wb5bU9FkmZ84WkkL37rKA,2,06,Fri
437648,2019-09-27,166,Chicken Noodle Soup (feat. Becky G),257406,United States,6wyr4ReB05D9sJB1Rsmcqo,0b1sIQumIAsNbqAoIClSpy,76IRLp7YzBVLKsat6Ro9ae,2,09,Fri
437649,2019-09-27,166,Chicken Noodle Soup (feat. Becky G),257406,United States,6wyr4ReB05D9sJB1Rsmcqo,4obzFoKoKRHIphyHzJ35G3,76IRLp7YzBVLKsat6Ro9ae,2,09,Fri
437690,2017-03-17,196,Dennis Rodman,166943,United States,73cAKC1NbxHuFPcQ4slGtl,2o8lOQRjzsSC8UdbNN88HN,5WS1g0cKtjfK6eDoSLdv7d,2,03,Fri


In [18]:
artist_name = data1[['Date', 'Track URI', 'No. of Collaborators', 'Collaborator Name']]
artist_name2 = artist_name.explode('Collaborator Name', ignore_index=True)
artist_name2[artist_name2['No. of Collaborators']!=1]

,Date,Track URI,No. of Collaborators,Collaborator Name
182,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,100 gecs
183,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Fall Out Boy
184,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Craig Owens
185,2020-07-10,5Mm2CJzNRiICC5MWRWQnBo,4,Nicole Dollanganger
186,2017-01-01,39pS70eeDvyCAF3t8NAlVV,3,2 Chainz
...,...,...,...,...
437644,2020-10-01,4iHSE5R1U8jf84tRn52xRt,2,Lil Baby
437648,2019-09-27,6wyr4ReB05D9sJB1Rsmcqo,2,j-hope
437649,2019-09-27,6wyr4ReB05D9sJB1Rsmcqo,2,Becky G
437690,2017-03-17,73cAKC1NbxHuFPcQ4slGtl,2,mansionz


In [19]:
data2 = artist_name2.merge(artist_uri2, how = 'left', 
                           on = ['Date', 'Track URI', 'No. of Collaborators'],
                           left_index=True, right_index=True)
data2['Collab No.'] = data2.groupby(['Track URI', 'Date']).cumcount() + 1

In [20]:
# data2[data2['No. of Collaborators']!=1]

### Generate variables that change on a artist-day basis

**Artist_first_onchart**: identify the first day an artist appears on the Top 200 chart.

**Artist_new_song**: generate a dummy variable that = 1 if this is the first time a track URI appears (Song_days_onchart == 1).

**Artist_new_collab**: generate a dummy variable that = 1 if this is the first time a track URI appears that is also a collab (No. of Collaborators != 1).

**Artist_new_solo**: generate a dummy variable that = 1 if this is the first time a track URI appears that is a solo (No. of Collaborators == 1).

**Artist_cumsum_songs**: calculate the cumulative sum of artist_new_song.

However, in the event that an artist has multiple songs on the chart on the same day, then this cumsum will be different for each song. To remedy this, we take the max cumsum per day for an artist to show how many songs an artist has on that day in total.

**Artist_cumu_songs**: the cumulative number of songs for an artist in a given day
(for each collaborator URI and date, find the max artist_cumsum_songs).

Analogously for Artist_cumu_collabs and Artist_cumu_solo.

Artist_cumu_songs should equal Artist_cumu_collabs + Artist_cumu_solo.

In [21]:
data2d = data2.merge(track_days, 
                      how= 'left', on = ['Date', 'Track URI'])

data2d['Artist_first_onchart'] = data2d.groupby(['Collaborator URI'])[['Date']].transform('min')
data2d['Artist_new_song'] = np.where(data2d['Song_days_onchart']==1, 
                                     1, 0)

data2d['Artist_new_collab'] = np.where((data2d['Artist_new_song']==1) &
                                      (data2d['No. of Collaborators']!=1),
                                       1, 0)
data2d['Artist_new_solo'] = np.where((data2d['Artist_new_song']==1) &
                                      (data2d['No. of Collaborators']==1),
                                       1, 0)

data2d['Artist_cumsum_songs'] = data2d.groupby(['Collaborator URI'])[['Artist_new_song']]\
.transform('cumsum')
data2d['Artist_cumu_songs'] = data2d.groupby(['Collaborator URI', 'Date'])[['Artist_cumsum_songs']]\
.transform('max')

data2d['Artist_cumsum_collab'] = data2d.groupby(['Collaborator URI'])[['Artist_new_collab']]\
.transform('cumsum')
data2d['Artist_cumu_collab'] = data2d.groupby(['Collaborator URI', 'Date'])[['Artist_cumsum_collab']]\
.transform('max')

data2d['Artist_cumsum_solo'] = data2d.groupby(['Collaborator URI'])[['Artist_new_solo']]\
.transform('cumsum')
data2d['Artist_cumu_solo'] = data2d.groupby(['Collaborator URI', 'Date'])[['Artist_cumsum_solo']]\
.transform('max')

In [22]:
# data2d

For each artist-day, find how many days the artist has been on the chart (cumulatively, not necessarily consecutively).

In [23]:
artist_days = data2d.loc[:, ['Collaborator URI', 'Date']]\
.sort_values(by=['Collaborator URI', 'Date'], ascending=[True,True]).drop_duplicates()
artist_days['Artist_days_onchart'] = artist_days.groupby(['Collaborator URI']).cumcount() + 1

artist_days

,Collaborator URI,Date,Artist_days_onchart
399078,002HSjuWsGMinkXTa7JcRp,2020-04-03,1
215451,00FQb4jTyendYWaN8pK0wa,2017-02-19,1
215452,00FQb4jTyendYWaN8pK0wa,2017-02-20,2
215453,00FQb4jTyendYWaN8pK0wa,2017-02-21,3
215454,00FQb4jTyendYWaN8pK0wa,2017-02-22,4
...,...,...,...
93782,7z5WFjZAIYejWy0NI5lv4T,2020-11-22,959
93783,7z5WFjZAIYejWy0NI5lv4T,2020-11-23,960
93786,7z5WFjZAIYejWy0NI5lv4T,2020-11-24,961
93789,7z5WFjZAIYejWy0NI5lv4T,2020-11-25,962


Merge data2e with artist days information.

In [24]:
data2e = data2d.merge(artist_days, how='left', on = ['Collaborator URI', 'Date'])

In [25]:
# data2e

Drop intermediary (dummy) variables not useful for EDAs/model building.

In [26]:
data3 = data2e.drop(columns = ['Artist_new_song', 'Artist_cumsum_songs',
                               'Artist_new_collab', 'Artist_cumsum_collab',
                               'Artist_new_solo', 'Artist_cumsum_solo',
                               'Date_diff', 'Date_lag',
                               'Song_new_streak'])
# data3

For each track URI-date, find the average characteristics for all artists in the collaboration:

**Collab_avg_days_onchart**: average number of days on the chart of all collaborators

**Collab_avg_cumu_songs**: average number of songs on the chart of all collaborators

**Collab_avg_cumu_collab**: average number of previous collaborations of all collaborators

**Collab_avg_cumu_solo**: average number of previous solo songs released of all collaborators

In [27]:
data3b = data3

data3b['Collab_avg_days_onchart'] = data3.groupby(['Track URI', 'Date'])[['Artist_days_onchart']].transform('mean')
data3b['Collab_avg_cumu_songs'] = data3.groupby(['Track URI', 'Date'])[['Artist_cumu_songs']].transform('mean')
data3b['Collab_avg_cumu_collab'] = data3.groupby(['Track URI', 'Date'])[['Artist_cumu_collab']].transform('mean')
data3b['Collab_avg_cumu_solo'] = data3.groupby(['Track URI', 'Date'])[['Artist_cumu_solo']].transform('mean')

In [28]:
data3b

,Date,Track URI,No. of Collaborators,Collaborator Name,Position,Track Name,Streams,Country,Collaborator URI,Album URI,Album_release_month,Album_release_dayweek,Collab No.,Album Release,Year_chart,Month_chart,Song_first_onchart,Song_days_since_first,Song_days_onchart,Song_days_since_release,Song_streak_id,Song_consec_day,Artist_first_onchart,Artist_cumu_songs,Artist_cumu_collab,Artist_cumu_solo,Artist_days_onchart,Collab_avg_days_onchart,Collab_avg_cumu_songs,Collab_avg_cumu_collab,Collab_avg_cumu_solo
0,2018-09-07,4nutwPQrK56fFmrAMgyPhz,1,$uicideBoy$,196,"10,000 Degrees",232152,United States,1VPmR4DJC1PlOtd0IADAO0,72I2i7wwU3Q7mJGxbNW12D,09,Fri,1,2018-09-07,2018,09,2018-09-07,0,1,0,1,1,2018-09-07,5,0,5,1,1.0,5.0,0.0,5.0
1,2018-09-07,7v2azTfke2BR57lh2HxPQo,1,$uicideBoy$,177,Bring Out Your Dead,247359,United States,1VPmR4DJC1PlOtd0IADAO0,72I2i7wwU3Q7mJGxbNW12D,09,Fri,1,2018-09-07,2018,09,2018-09-07,0,1,0,1,1,2018-09-07,5,0,5,1,1.0,5.0,0.0,5.0
2,2018-09-07,0fyBYsrmpihh1mfalssDlB,1,$uicideBoy$,146,King Tulip,270299,United States,1VPmR4DJC1PlOtd0IADAO0,72I2i7wwU3Q7mJGxbNW12D,09,Fri,1,2018-09-07,2018,09,2018-09-07,0,1,0,1,1,2018-09-07,5,0,5,1,1.0,5.0,0.0,5.0
3,2018-09-07,6VFKlX5qzxwmIiezqeqNYG,1,$uicideBoy$,190,Meet Mr. NICEGUY,236054,United States,1VPmR4DJC1PlOtd0IADAO0,72I2i7wwU3Q7mJGxbNW12D,09,Fri,1,2018-09-07,2018,09,2018-09-07,0,1,0,1,1,2018-09-07,5,0,5,1,1.0,5.0,0.0,5.0
4,2018-09-07,53AiGAa0Qi2VbX7eUpur1U,1,$uicideBoy$,140,Nicotine Patches,279299,United States,1VPmR4DJC1PlOtd0IADAO0,72I2i7wwU3Q7mJGxbNW12D,09,Fri,1,2018-09-07,2018,09,2018-09-07,0,1,0,1,1,2018-09-07,5,0,5,1,1.0,5.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437768,2017-11-10,1YqcGlCHNquxBhlUZsjhMT,1,,182,NaN,205945,United States,2uwXb02RFgtGOr6s6XMWlB,2oxjmU5c2mZzc7mSQqWTE9,11,Tue,1,2017-11-07,2017,11,2017-11-09,1,2,3,1,2,2017-11-09,1,0,1,2,2.0,1.0,0.0,1.0
437769,2017-11-11,1YqcGlCHNquxBhlUZsjhMT,1,,151,NaN,200095,United States,2uwXb02RFgtGOr6s6XMWlB,2oxjmU5c2mZzc7mSQqWTE9,11,Tue,1,2017-11-07,2017,11,2017-11-09,2,3,4,1,3,2017-11-09,1,0,1,3,3.0,1.0,0.0,1.0
437770,2017-11-12,1YqcGlCHNquxBhlUZsjhMT,1,,160,NaN,176687,United States,2uwXb02RFgtGOr6s6XMWlB,2oxjmU5c2mZzc7mSQqWTE9,11,Tue,1,2017-11-07,2017,11,2017-11-09,3,4,5,1,4,2017-11-09,1,0,1,4,4.0,1.0,0.0,1.0
437771,2017-11-13,1YqcGlCHNquxBhlUZsjhMT,1,,170,NaN,190496,United States,2uwXb02RFgtGOr6s6XMWlB,2oxjmU5c2mZzc7mSQqWTE9,11,Tue,1,2017-11-07,2017,11,2017-11-09,4,5,6,1,5,2017-11-09,1,0,1,5,5.0,1.0,0.0,1.0


Set index using Date, Track URI, and the ID of the collaborator in the track (Collab No.).
In the event that we need to pivot the dataframe to wide format (so that each artist's information is on a separate column), we can unstack using the Collab No.

In [29]:
data3c = data3b\
.set_index(['Date', 'Track URI', 'Collab No.'], append=False)
# data3c.loc[data3c['No. of Collaborators']!=1,:]

In [30]:
# data3d = data3c.unstack(level=2)

In [31]:
# data3e = data3d
# data3e.columns = [' '.join(col).strip() for col in data3e.columns.values]
# data3e

In [32]:
# data3c.loc[data3c['No. of Collaborators']!=1,:]\
# .to_csv('../output/2021.01.01 US Collab Wide.csv')